In [ ]:
import datetime as datetime
import time as time

import pandas as pd
import talib
from backtesting import Strategy, Backtest
from backtesting.lib import crossover, resample_apply

In [ ]:
#IMPORT LIST OF SP500 STOCKS HERE IN THE TICKER VARIABLE TO RUN TESTS ON ALL OF THE INDIVIDUAL
#STOCKS

#Ticker in url
ticker = 'AAPL'

#Timeperiods of data set "Y/M/D/time"
#this will help when selecting the desired dates and will pull the data set from yahoo finance.
period1 = int(time.mktime(datetime.datetime(2013, 1, 1, 23, 59).timetuple()))
period2 = int(time.mktime(datetime.datetime(2023, 1, 12, 23, 59).timetuple()))
interval = '1d' # 1wk, 1m

#Yahoo Finance url
url = f'https://query1.finance.yahoo.com/v7/finance/download/{ticker}?period1={period1}&period2={period2}&interval={interval}&events=history&includeAdjustedClose=true'

#defining df to backtesting requirements
df = pd.read_csv(url)
columns = ['Date', 'Open', 'High', 'Low', 'Close', 'adj close', 'Volume']
df.columns = columns

#index df to date time index to fit backtesting.py
#df requirements
df = df.set_index(pd.DatetimeIndex(df['Date'].values))

#dropping columns that are not necessary for backtesting.py format
df.drop('Date', inplace=True, axis=1)
df.drop('adj close', inplace=True, axis=1)
df

In [ ]:
#the overall trading strategy function
class MACD(Strategy):

    #defines the premade trading parameters imported from talib
    #(trading parameter, data column being used, trading window)
    def init(self):

        #MACD variables
        self.macd, self.macdsignal, self.macdhist = self.I(talib.MACD, self.data.Close, fastperiod=12, slowperiod=26, signalperiod=9)
        #A comparison from the TA-lib directory
        #macd, macdsignal, macdhist = MACD(close, fastperiod=12, slowperiod=26, signalperiod=9)

    def next(self):

        if crossover(self.macdsignal, self.macd):
            #if this statement is true the below command signals a sell.
            self.position.close()
            # self.sell()

        elif crossover(self.macd, self.macdsignal):
            #buy command
            # self.position.close()
            self.buy()


#bt variable runs the backtest dependant on the data, strategy, and cash
#other parameters can be added to more complex strategies. Refer to
#backtesting.py on github
bt = Backtest(df, MACD, cash = 10_000)
stats = bt.run()
stats
#the plotting function does not work in python 3.8.7 so it needs to be
#run in a earlier python like python 3.6 to graph the trades
#bt.plot()

In [ ]:
bt.plot()

In [ ]:
stats.tail()
stats['_trades']

In [ ]:
#the overall trading strategy function
class Stochastic(Strategy):

    #defines the premade trading parameters imported from talib
    #(trading parameter, data column being used, trading window)
    def __init__(self, broker, data, params):
        super().__init__(broker, data, params)
        self.stochd = None
        self.stochk = None

    def init(self):
        #STOCH variables
        self.stochk, self.stochd = self.I(talib.STOCH, self.data.High, self.data.Low, self.data.Close, fastk_period=14, slowk_period=3, slowd_period=3)

        #slowk, slowd = STOCH(high, low, close, fastk_period=5, slowk_period=3, slowk_matype=0, slowd_period=3, slowd_matype=0)
    def next(self):

        if self.stochk > 80 and self.stochd > 80 and crossover(self.stochk, self.stochd):
            #if this statement is true the below command signals a sell.
            self.position.close()
        elif self.stochk < 20 and self.stochd < 20 and crossover(self.stochd, self.stochk):
            #buy command
            self.buy()

#bt variable runs the backtest dependant on the data, strategy, and cash
#other parameters can be added to more complex strategies. Refer to
#backtesting.py on github
bt = Backtest(df, Stochastic)
stats = bt.run()
stats


In [ ]:
stats['_trades']

## MACD and Stochastic: A Double-Cross Strategy

https://www.investopedia.com/articles/trading/08/macd-stochastic-double-cross.asp#:~:text=The%20Strategy,-First%2C%20look%20for&text=When%20applying%20the%20stochastic%20and,days%20of%20placing%20your%20trade.

### The Strategy

First, look for the bullish crossovers to occur within two days of each other. When applying the stochastic and MACD double-cross strategy, ideally, the crossover occurs below the 50-line on the stochastic to catch a longer price move. And preferably, you want the histogram value to already be or move higher than zero within two days of placing your trade.

Also note the MACD must cross slightly after the stochastic, as the alternative could create a false indication of the price trend or place you in a sideways trend.

Finally, it is safer to trade stocks trading above their 200-day moving averages, but it is not an absolute necessity.



In [ ]:
#the overall trading strategy function
class DoubleCross(Strategy):

    #defines the premade trading parameters imported from talib
    #(trading parameter, data column being used, trading window)
    def __init__(self, broker, data, params):
        super().__init__(broker, data, params)
        self.stochd = None
        self.stochk = None
        self.macdhist = None
        self.macdsignal = None
        self.macd = None
        self.db_lookback = -4

    def init(self):
        #STOCH variables
        self.macd, self.macdsignal, self.macdhist =  self.I(talib.MACD, self.data.Close, fastperiod=12, slowperiod=26, signalperiod=9)
        self.stochk, self.stochd = resample_apply("D", talib.STOCH, self.data.High, self.data.Low, self.data.Close, fastk_period=14, slowk_period=3, slowd_period=3)

    def next(self):
        if crossover(self.macdsignal, self.macd):
            #if this statement is true the below command signals a sell.
            for i in range(-1, self.db_lookback, -1):
                if 80 < self.stochk[i] < self.stochd[i]:
                    self.position.close()
                    break
        elif crossover(self.macd, self.macdsignal):
            for i in range(-1, self.db_lookback, -1):
                if 50 > self.stochk[i] > self.stochd[i]:
                    self.buy()
                    break


bt = Backtest(df, DoubleCross, cash = 10_000, exclusive_orders=True)
stats = bt.run()
stats
#the plotting function does not work in python 3.8.7 so it needs to be
#run in a earlier python like python 3.6 to graph the trades
#bt.plot()

In [ ]:
bt.plot()

In [ ]:
stats.tail()
stats['_trades']

In [ ]:

#the plotting function does not work in python 3.8.7 so it needs to be
#run in a earlier python like python 3.6 to graph the trades
bt.plot()